In [1]:
import sys
sys.path.append('../../')
sys.path.append('../')
import time
import wd

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

replace success!


In [3]:
from torch.profiler import profile, record_function, ProfilerActivity
import time
import torch._dynamo as dynamo


In [4]:
def gen_pattern_replace_and_matcher_for_MLP(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  key_node_name,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[key_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.embed_output_dim = shape_info[1]
          self.mlp = nn.Linear(shape_info[0],shape_info[1])


      def forward(self,x):
          x = self.embed(x).view(-1,self.embed_output_dim)
          return self.mlp(x)
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
  linear_node = node_map[pattern_env['mlp']]
  linear_node_module = utils.get_target_mod(traced,linear_node.target)
  linear_node_weight = linear_node_module.weight.data
  linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]
      self.redency_weight_len = self.embed_dim * redency_part_slice[1].stop
      redency_weight = linear_node_weight[:,:self.redency_weight_len]
      unredency_weight = linear_node_weight[:,self.redency_weight_len:]
      self.unredency_weight_len = unredency_weight.shape[1]
      self.redency_linear = nn.Linear(redency_weight.shape[1],redency_weight.shape[0])
      self.redency_linear.weight.data.copy_(redency_weight)
      self.redency_linear.bias.data.copy_(linear_node_bias)

      self.unredency_linear = nn.Linear(unredency_weight.shape[1],unredency_weight.shape[0],bias=False)
      self.unredency_linear.weight.data.copy_(unredency_weight)

      

    def forward(self,x):
      redency_part = x[redency_part_slice] 
      unredency_part = x[unredency_part_slice] 
      return self.redency_linear(self.embed(redency_part).view(-1,self.redency_weight_len)) + self.unredency_linear(self.embed(unredency_part).view(-1,self.unredency_weight_len))
      # return unredency_sum
    
  
  return pattern,ReplacementClass(),_match

In [5]:
def workload_wdl(num_field, prefix,dim = 64,l = [1024,512,256]):
  print(f"now gen workload of wdl with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}")
  wdl_model_ori = wd.WideAndDeepModel([100 for i in range(num_field)],dim,l,0.1)
  ori_traced = symbolic_trace(wdl_model_ori)
  
  wdl_model_modify = wd.WideAndDeepModel([100 for i in range(num_field)],dim,l,0.1)
  modify_traced = symbolic_trace(wdl_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_MLP(modify_traced,
                                                                      (0,slice(None,prefix,None)),(slice(None,None,None),slice(prefix,None,None)),
                                                                      "mlp_mlp_0")
  matches = subgraph_rewriter.replace_pattern_with_filters(modify_traced, pattern, replace,[match])
  return ori_traced,modify_traced

In [6]:
def calculate_mean_and_variance_manual(data):
    n = len(data)
    mean = sum(data) / n
    variance = sum((x - mean) ** 2 for x in data) / n
    return mean, variance

In [7]:
def gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 32, workload_func = workload_wdl):
  def run(model):
    t = torch.randint(low=0, high=88, size=(batch,num_field), dtype=torch.long).cuda()
    model.eval()
    traced_model = torch.jit.trace(model.cuda(), t)
    compiled_model = torch.compile(traced_model, backend="inductor")
    compiled_model.eval()
    for i in range(10):
       soutput = compiled_model(t)
    torch.cuda.synchronize()
    start = time.time()
    for i in range(100):
        soutput = compiled_model(t)
    end = time.time()        
    print(f"cal time : {(end - start)/100 * 1000}")
  print(f"now gen workload of fm with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}, batch :{batch}")
  ori, modify = workload_func(num_field,prefix,dim)
  run(ori)
  run(modify)

In [9]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 110, prefix: 50, batch :1024
now gen workload of wdl with config: dim: 32, num_field: 110, prefix: 50
cal time : 0.3018379211425781
cal time : 0.3959083557128906


In [10]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 110, prefix: 50, batch :2048
now gen workload of wdl with config: dim: 32, num_field: 110, prefix: 50
cal time : 0.47260522842407227
cal time : 0.4111790657043457


In [11]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 110, prefix: 50, batch :4096
now gen workload of wdl with config: dim: 32, num_field: 110, prefix: 50
cal time : 0.9473919868469238
cal time : 0.8221173286437988


In [12]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 110, prefix: 50, batch :1024
now gen workload of wdl with config: dim: 64, num_field: 110, prefix: 50
cal time : 0.4098963737487793
cal time : 0.39061546325683594


In [13]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 110, prefix: 50, batch :2048
now gen workload of wdl with config: dim: 64, num_field: 110, prefix: 50
cal time : 0.8155655860900879
cal time : 0.6353068351745605


In [14]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 110, prefix: 50, batch :4096
now gen workload of wdl with config: dim: 64, num_field: 110, prefix: 50
cal time : 1.7202115058898926
cal time : 1.236124038696289


In [16]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 170, prefix: 145, batch :1024
now gen workload of wdl with config: dim: 32, num_field: 170, prefix: 145
cal time : 0.3360939025878906
cal time : 0.3961062431335449


In [17]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 170, prefix: 145, batch :2048
now gen workload of wdl with config: dim: 32, num_field: 170, prefix: 145
cal time : 0.6471657752990723
cal time : 0.39620161056518555


In [18]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 170, prefix: 145, batch :4096
now gen workload of wdl with config: dim: 32, num_field: 170, prefix: 145
cal time : 1.366870403289795
cal time : 0.45939207077026367


In [20]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 170, prefix: 145, batch :1024
now gen workload of wdl with config: dim: 64, num_field: 170, prefix: 145
cal time : 0.6141996383666992
cal time : 0.3941512107849121


In [21]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 170, prefix: 145, batch :2048
now gen workload of wdl with config: dim: 64, num_field: 170, prefix: 145
cal time : 1.1839914321899414
cal time : 0.40170907974243164


In [22]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 170, prefix: 145, batch :4096
now gen workload of wdl with config: dim: 64, num_field: 170, prefix: 145
cal time : 2.5402045249938965
cal time : 0.6920075416564941


In [24]:
gen_and_test(num_field = 100,prefix = 10, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 10, batch :4096
now gen workload of wdl with config: dim: 32, num_field: 100, prefix: 10
cal time : 0.8940577507019043
cal time : 1.0836386680603027


In [25]:
gen_and_test(num_field = 100,prefix = 20, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 20, batch :4096
now gen workload of wdl with config: dim: 32, num_field: 100, prefix: 20
cal time : 0.8911347389221191
cal time : 0.9838080406188966


In [26]:
gen_and_test(num_field = 100,prefix = 30, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 30, batch :4096
now gen workload of wdl with config: dim: 32, num_field: 100, prefix: 30
cal time : 0.8811545372009277
cal time : 0.9004735946655273


In [27]:
gen_and_test(num_field = 100,prefix = 40, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 40, batch :4096
now gen workload of wdl with config: dim: 32, num_field: 100, prefix: 40
cal time : 0.8764505386352539
cal time : 0.8038806915283203


In [28]:
gen_and_test(num_field = 100,prefix = 50, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 50, batch :4096
now gen workload of wdl with config: dim: 32, num_field: 100, prefix: 50
cal time : 0.8893251419067383
cal time : 0.7136321067810059


In [29]:
gen_and_test(num_field = 100,prefix = 60, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 60, batch :4096
now gen workload of wdl with config: dim: 32, num_field: 100, prefix: 60
cal time : 0.8905410766601562
cal time : 0.6316924095153809


In [30]:
gen_and_test(num_field = 100,prefix = 70, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 70, batch :4096
now gen workload of wdl with config: dim: 32, num_field: 100, prefix: 70
cal time : 0.8832216262817383
cal time : 0.49904584884643555


In [31]:
gen_and_test(num_field = 100,prefix = 80, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 80, batch :4096
now gen workload of wdl with config: dim: 32, num_field: 100, prefix: 80
cal time : 0.8940815925598145
cal time : 0.39601802825927734


In [32]:
gen_and_test(num_field = 100,prefix = 90, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 90, batch :4096
now gen workload of wdl with config: dim: 32, num_field: 100, prefix: 90
cal time : 0.9016704559326172
cal time : 0.39067983627319336


In [33]:
gen_and_test(num_field = 100,prefix = 99, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 99, batch :4096
now gen workload of wdl with config: dim: 32, num_field: 100, prefix: 99
cal time : 0.8880877494812012
cal time : 0.3796529769897461


In [6]:
def genWorkload(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64):
  ori_model_name = f'/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/wdl/wdl_{batch}_{num_field}_{prefix}_{dim}_ori.onnx'
  modify_model_name = f'/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/wdl/wdl_{batch}_{num_field}_{prefix}_{dim}_modify.onnx'
  ori, modify = workload_wdl(num_field = num_field,prefix = prefix,  dim = dim)
  torch.onnx.export(ori,               # 模型 being run
                  torch.randint(low=0, high=20, size=(batch,num_field), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
                  ori_model_name,        # 导出文件的文件名
                  export_params=True, # 如果设置为True，则参数也会被导出。注意某些情况下参数可能无法被导出。
                  opset_version=10,   # ONNX版本
                  do_constant_folding=True,  # 是否执行常量折叠以优化模型
                  input_names = ['input'],   # 输入的名称
                  output_names = ['output'], # 输出的名称
                  )
  torch.onnx.export(modify,               # 模型 being run
                  torch.randint(low=0, high=20, size=(batch,num_field), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
                  modify_model_name,        # 导出文件的文件名
                  export_params=True, # 如果设置为True，则参数也会被导出。注意某些情况下参数可能无法被导出。
                  opset_version=10,   # ONNX版本
                  do_constant_folding=True,  # 是否执行常量折叠以优化模型
                  input_names = ['input'],   # 输入的名称
                  output_names = ['output'], # 输出的名称
                  )

In [8]:
dims= [32]
batches = [1024,2048,4096]
num_field_and_prefixs = [(34 * 5,29*5),(22 * 5,10 * 5)]

In [9]:
for dim in dims:
  for batch in batches:
    for num_field,prefix in num_field_and_prefixs:
      genWorkload(num_field=num_field,prefix=prefix,batch=batch,dim=dim)

now gen workload of wdl with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1314: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of wdl with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1314: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of wdl with config: dim: 32, num_field: 170, prefix: 145


...

/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1314: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of wdl with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1314: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of wdl with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1314: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


In [ ]:
dims= [32]
batches = [1024,2048,4096]
num_field_and_prefixs = [(34 ,29),(22 ,10 ),(34 * 5,29*5),(22 * 5,10 * 5)]

In [8]:
for dim in dims:
  for batch in batches:
    for num_field,prefix in num_field_and_prefixs:
      genWorkload(num_field=num_field,prefix=prefix,batch=batch,dim=dim)

now gen workload of wdl with config: dim: 32, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(
/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of wdl with config: dim: 32, num_field: 22, prefix: 10
now gen workload of wdl with config: dim: 32, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(
/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of wdl with config: dim: 32, num_field: 22, prefix: 10
now gen workload of wdl with config: dim: 32, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(
/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of wdl with config: dim: 32, num_field: 22, prefix: 10
